In [91]:
import dill

data_directory =  "/Users/simon.hughes/Google Drive/Phd/Data/"
root_folder = data_directory + "CoralBleaching/Thesis_Dataset/"
rnn_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"

fname = rnn_predictions_folder + "essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(fname, "rb") as f:
    pred_tagged_essays_cb = dill.load(f)

In [92]:
from load_data import load_process_essays
from window_based_tagger_config import get_config
from Settings import Settings

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
test_config = get_config(test_folder)
tagged_essays_test_cb = load_process_essays(**test_config)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
226 files found
226 essays processed


In [93]:
root_folder = data_directory + "SkinCancer/Thesis_Dataset/"
rnn_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"

fname = rnn_predictions_folder + "essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(fname, "rb") as f:
    pred_tagged_essays_sc = dill.load(f)

In [94]:
root_folder = settings.data_directory + "SkinCancer/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
test_config = get_config(test_folder)
tagged_essays_test_sc = load_process_essays(**test_config)

218 files found
218 essays processed


In [16]:
cb_collections = [
    'CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN',
    'CR_CB_STACKED_VD',
    'CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD'
]
test_collections = ["TEST_" + c.replace("HYPER_PARAM_","") for c in cb_collections]

In [134]:
import pymongo
import numpy as np

client = pymongo.MongoClient()
db = client.metrics_causal

def print_macro_table_row(coll, algo, crels, db=db):
    best_f1 = -1
    best_tuple = None
    for row in db[coll].find({}):
        precision, recall,ncodes = [],[],[]        
        for k in row.keys():
            #TODO - fix this to be more specific
            if k in crels:
                code, prec, rec = k, row[k]["precision"], row[k]["recall"]
                precision.append(prec)
                recall.append(rec)
                ncodes.append(row[k]["num_codes"])
        mprec = np.mean(precision)
        mrec =  np.mean(recall)
        count = sum(ncodes)
        if mprec > 0 or mrec > 0:
            est_mf1 = (2 * mprec * mrec) / (mprec + mrec)
            #For some reason, the MONGO macro F1 does not match the expected from all the classes
            #let's compute this here so that the math works (in case they compute it from the underlying macro recall and prec)
            print("{algo} &\t{count}\t&\t{mf1:.3f}\t\t&\t{rec:.3f}\t\t&\t{prec:.3f} \\\\".format(
                algo=algo.ljust(25), count=count, mf1=est_mf1, prec=mprec, rec=mrec))
        else:
            print("{algo} &\t{count}\t&\t{mf1:.3f}\t\t&\t{rec:.3f}\t\t&\t{prec:.3f} \\\\".format(
                algo=algo.ljust(25), count=count, mf1=0.0, prec=0.0, rec=0.0))

In [146]:
def print_micro_table_row(coll, algo, crels, db=db):
    best_f1 = -1
    best_tuple = None
    for row in db[coll].find({}):
        tp,fp,fn, ncodes = 0,0,0,0
        for k in row.keys():
            #TODO - fix this to be more specific
            if k in crels:
                metrics = row[k]
                tp += metrics["tp"]
                fp += metrics["fp"]
                fn += metrics["fn"]
                ncodes += metrics["num_codes"]
        if tp + fp == 0:
            mprec = 0
        else:
            mprec = tp / (tp + fp)
        
        if tp + fn == 0:
            mrec = 0
        else:
            mrec = tp / (tp + fn)
       
        if mprec > 0 or mrec > 0:
            est_mf1 = (2 * mprec * mrec) / (mprec + mrec)
            #For some reason, the MONGO macro F1 does not match the expected from all the classes
            #let's compute this here so that the math works (in case they compute it from the underlying macro recall and prec)
            print("{algo} &\t{count}\t&\t{mf1:.3f}\t\t&\t{rec:.3f}\t\t&\t{prec:.3f} \\\\".format(
                algo=algo.ljust(25), count=ncodes, mf1=est_mf1, prec=mprec, rec=mrec))
        else:
            print("{algo} &\t{count}\t&\t{mf1:.3f}\t\t&\t{rec:.3f}\t\t&\t{prec:.3f} \\\\".format(
                algo=algo.ljust(25), count=ncodes, mf1=0.0, prec=0.0, rec=0.0))

In [167]:
def print_code_metrics(coll, crels, db=db):
    best_f1 = -1
    best_tuple = None
    for row in db[coll].find({}):
        tp,fp,fn, ncodes = 0,0,0,0
        for k in row.keys():
            #TODO - fix this to be more specific
            if k in crels:
                metrics = row[k]
                tp = metrics["tp"]
                tn = metrics["tn"]
                fp = metrics["fp"]
                fn = metrics["fn"]
                ncodes = metrics["num_codes"]
                print("{code} \t tp={tp} \t tn={tn} \t fp={fp} \t fn={fn} \t codes={codes}".format(
                    code=k.ljust(25), tp=tp, tn=tn, fp=fp, fn=fn, codes=ncodes))

In [168]:
def get_causal_algo_name(coll):
    if "RNN" in coll:
        return "RNN Word Tagging Model"
    if "SHIFT" in coll:
        return "Shift-Reduce Parser"
    if "STACK" in coll:
        return "Stacking Model"
    return "xxx"

In [169]:
from crel_helper import get_tag_freq

tag_freq_train = get_tag_freq(pred_tagged_essays_cb, [])
crel_tags_train = list((t for t in tag_freq_train.keys() if ("->" in t) and
                not "Anaphor" in t and
                not "other" in t and
                not "rhetorical" in t and
                not "factor" in t and
                1 == 1
                ))

tag_freq_test = get_tag_freq(tagged_essays_test_cb, [])
crel_tags_test = list((t for t in tag_freq_test.keys() if ("->" in t) and
                not "Anaphor" in t and
                not "other" in t and
                not "rhetorical" in t and
                not "factor" in t and
                1 == 1
                ))
test_crels_cb = set(crel_tags_test) - set(crel_tags_train)
test_crels_cb

{'Causer:5->Result:11',
 'Causer:6->Result:5',
 'Causer:7->Result:1',
 'Causer:7->Result:13'}

In [170]:
from crel_helper import get_tag_freq

tag_freq_train = get_tag_freq(pred_tagged_essays_sc, [])
crel_tags_train = list((t for t in tag_freq_train.keys() if ("->" in t) and
                not "Anaphor" in t and
                not "other" in t and
                not "rhetorical" in t and
                not "factor" in t and
                1 == 1
                ))

tag_freq_test = get_tag_freq(tagged_essays_test_sc, [])
crel_tags_test = list((t for t in tag_freq_test.keys() if ("->" in t) and
                not "Anaphor" in t and
                not "other" in t and
                not "rhetorical" in t and
                not "factor" in t and
                1 == 1
                ))
test_crels_sc = set(crel_tags_test) - set(crel_tags_train)
test_crels_sc

{'Causer:11->Result:4', 'Causer:5->Result:12'}

## Coral Bleaching

In [171]:
for coll in test_collections[-1:]:
    print_code_metrics(coll, crels=test_crels_cb, db = client.metrics_causal)

Causer:5->Result:11       	 tp=0.0 	 tn=1917.0 	 fp=0.0 	 fn=1.0 	 codes=1
Causer:6->Result:5        	 tp=1.0 	 tn=1917.0 	 fp=0.0 	 fn=0.0 	 codes=1
Causer:7->Result:13       	 tp=0.0 	 tn=1916.0 	 fp=1.0 	 fn=1.0 	 codes=1
Causer:7->Result:1        	 tp=0.0 	 tn=1917.0 	 fp=0.0 	 fn=1.0 	 codes=1


In [150]:
for coll in test_collections:
    name = get_causal_algo_name(coll)
    print_micro_table_row(coll, name, crels=test_crels_cb, db = client.metrics_causal)

RNN Word Tagging Model    &	4	&	0.000		&	0.000		&	0.000 \\
Stacking Model            &	4	&	0.000		&	0.000		&	0.000 \\
Shift-Reduce Parser       &	4	&	0.333		&	0.250		&	0.500 \\


In [151]:
for coll in test_collections:
    name = get_causal_algo_name(coll)
    print_macro_table_row(coll, name, crels=test_crels_cb, db = client.metrics_causal)

RNN Word Tagging Model    &	4	&	0.000		&	0.000		&	0.000 \\
Stacking Model            &	4	&	0.000		&	0.000		&	0.000 \\
Shift-Reduce Parser       &	4	&	0.250		&	0.250		&	0.250 \\


## Skin Cancer

In [173]:
for coll in test_collections[-1:]:
    coll = coll.replace("CB", "SC")
    print_code_metrics(coll, crels=test_crels_sc, db = client.metrics_causal)

Causer:5->Result:12       	 tp=0.0 	 tn=2096.0 	 fp=0.0 	 fn=1.0 	 codes=1
Causer:11->Result:4       	 tp=0.0 	 tn=2096.0 	 fp=0.0 	 fn=1.0 	 codes=1


In [152]:
for coll in test_collections:
    coll = coll.replace("CB", "SC")
    #print(test_crels_sc)
    name = get_causal_algo_name(coll)
    print_micro_table_row(coll, name, crels=test_crels_sc, db = client.metrics_causal)

RNN Word Tagging Model    &	2	&	0.000		&	0.000		&	0.000 \\
Stacking Model            &	2	&	0.000		&	0.000		&	0.000 \\
Shift-Reduce Parser       &	2	&	0.000		&	0.000		&	0.000 \\


In [153]:
for coll in test_collections:
    coll = coll.replace("CB", "SC")
    name = get_causal_algo_name(coll)
    print_macro_table_row(coll, name, crels=test_crels_sc, db = client.metrics_causal)

RNN Word Tagging Model    &	2	&	0.000		&	0.000		&	0.000 \\
Stacking Model            &	2	&	0.000		&	0.000		&	0.000 \\
Shift-Reduce Parser       &	2	&	0.000		&	0.000		&	0.000 \\
